In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.datasets import fetch_20newsgroups
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
class DBSCAN:
    
    def __init__(self, X, eps, min_pts, distances=0, precomputed=False):
        self.eps = eps
        self.min_pts = min_pts
        self.X = X
        self.n_rows = X.shape[0]
        
        #  0 - unclassified
        # -1 - Noise
        # >0 - cluster id the point belngs to
        self.lable = np.zeros(self.n_rows, dtype='int16')
        
        # precompute distance matrix
        if not precomputed:
            self.distances = euclidean_distances(self.X)
        else:
            self.distances = distances
            
    def rangeQuery(self, pid):
        return np.where(self.distances[pid] <= self.eps)[0]
    
    def computeGini(self):
        Mj = self.conf_mat.sum(axis=1, keepdims=True)
        gj = 1 - ((self.conf_mat/Mj)**2).sum(axis=1, keepdims=True)
        return (sum(gj*Mj)/sum(Mj))[0]
    
    def computePurity(self):
        Pj = self.conf_mat.max(axis=1)
        Mj = self.conf_mat.sum(axis=1)
        return sum(Pj)/sum(Mj)
        
    def evaluateConfusion(self, y):
        c = len(np.unique(self.lable))
        d = len(np.unique(y))
        self.conf_mat = np.zeros(shape=(c - 1, d))
        
        # for noise (also change dimention of conf_mat)
        # self.conf_mat[0] = np.bincount(y[np.where(self.lable == -1)[0]].ravel(), minlength=d)
        
        # for clusters
        for cluster_id in range(1, c):
            true_labels = y[np.where(self.lable == cluster_id)[0]].ravel()
            self.conf_mat[cluster_id - 1] = np.bincount(true_labels, minlength=d)
    
    def fit(self):
        c_id = 0
        
        for n in range(self.n_rows):
            if self.lable[n] != 0:
                continue
            
            # get neighbours for point
            neighbours = self.rangeQuery(n)
            
            # density check
            if len(neighbours) < self.min_pts:
                self.lable[neighbours] = -1
                continue
            
            c_id += 1
            self.lable[n] = c_id
            
            # grow cluster
            idx = 0
            while idx < len(neighbours):
                _n = neighbours[idx]
                idx += 1
                
                # check if already processed
                if self.lable[_n] > 0:
                    continue
                    
                # add point to cluster
                self.lable[_n] = c_id
                _neighbours = self.rangeQuery(_n)
                
                # if core point, add to original neighbours
                if len(_neighbours) >= self.min_pts:
                    neighbours = np.concatenate((neighbours, _neighbours))

## FASHION

In [3]:
fashion_df = pd.read_csv('../data/fashion/fashion-mnist_train.csv')
y = fashion_df['label']
del fashion_df['label']
fashion_X = fashion_df.values
del fashion_df

In [4]:
np.bincount(y)

array([6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000])

Sample points and shuffle

In [4]:
label_idx = {}
sample_size = 600
for idx in range(len(np.unique(y))):
    label_idx[idx] = np.where(y == idx)[0]

np.random.seed(42)
for idx in range(len(label_idx)):
    label_idx[idx] = np.random.choice(label_idx[idx], sample_size, replace=False)

fashion_sub = np.empty((sample_size * len(label_idx), fashion_X.shape[1]))
y_sub = np.empty((sample_size * len(label_idx), 1), dtype='int16')
for idx in range(len(label_idx)):
    start = idx * sample_size
    end = (idx + 1) * sample_size
    fashion_sub[start:end] = fashion_X[label_idx[idx]]
    y_sub[start:end] = idx

s = np.arange(sample_size * len(label_idx))
np.random.shuffle(s)
fashion_sub = fashion_sub[s]
y_sub = y_sub[s]

DBSCAN on fashion

In [5]:
# estimate initial eps
d = euclidean_distances(fashion_sub)

In [6]:
np.mean(np.mean(np.sort(d, axis=1)[:, 2:5], axis=1))

1165.6730333644657

In [12]:
for eps in range(900, 1400, 100):
    print('\neps={0}'.format(eps))
    for min_pts in range(3, 5):
        print('\tminpts={0}'.format(min_pts))
        fashion_db = DBSCAN(X=fashion_sub, eps=eps, min_pts=min_pts, distances=d, precomputed=True)
        fashion_db.fit()
        print('\t', Counter(fashion_db.lable))


eps=900
	minpts=3
	 Counter({-1: 4423, 1: 885, 4: 238, 9: 52, 3: 48, 10: 45, 8: 37, 7: 29, 13: 27, 23: 14, 27: 14, 2: 13, 15: 11, 31: 11, 18: 10, 37: 10, 16: 7, 5: 6, 6: 6, 11: 6, 21: 6, 38: 6, 28: 5, 22: 5, 24: 5, 29: 5, 33: 5, 39: 5, 40: 5, 14: 4, 46: 4, 19: 4, 20: 4, 30: 4, 35: 4, 42: 4, 12: 3, 17: 3, 25: 3, 26: 3, 32: 3, 34: 3, 36: 3, 41: 3, 43: 3, 44: 3, 45: 3})
	minpts=4
	 Counter({-1: 4616, 1: 826, 4: 230, 10: 43, 9: 39, 3: 38, 8: 35, 7: 25, 18: 25, 21: 12, 2: 11, 15: 11, 12: 11, 14: 10, 23: 10, 16: 8, 6: 6, 24: 6, 17: 5, 22: 5, 25: 5, 26: 5, 11: 4, 19: 4, 20: 4, 5: 3, 13: 3})

eps=1000
	minpts=3
	 Counter({-1: 3456, 1: 2314, 13: 49, 9: 37, 3: 20, 14: 8, 10: 6, 11: 6, 18: 6, 25: 6, 6: 5, 30: 5, 19: 5, 29: 5, 27: 4, 7: 4, 15: 4, 31: 4, 33: 4, 32: 4, 2: 3, 4: 3, 5: 3, 8: 3, 12: 3, 16: 3, 17: 3, 20: 3, 21: 3, 22: 3, 23: 3, 24: 3, 26: 3, 28: 3, 34: 3, 35: 3})
	minpts=4
	 Counter({-1: 3684, 1: 2053, 3: 151, 8: 32, 9: 24, 7: 20, 2: 13, 6: 8, 5: 4, 11: 4, 4: 3, 10: 2, 12: 2})

eps=110

In [13]:
for eps in range(1200, 1310, 10):
    print('\neps={0}'.format(eps))
    for min_pts in range(3, 5):
        print('\tminpts={0}'.format(min_pts))
        fashion_db = DBSCAN(X=fashion_sub, eps=eps, min_pts=min_pts, distances=d, precomputed=True)
        fashion_db.fit()
        print('\t', Counter(fashion_db.lable))


eps=1200
	minpts=3
	 Counter({1: 4077, -1: 1862, 2: 16, 3: 11, 6: 8, 8: 6, 5: 5, 4: 4, 10: 4, 7: 4, 9: 3})
	minpts=4
	 Counter({1: 3973, -1: 1979, 7: 11, 2: 9, 3: 9, 6: 8, 5: 5, 4: 4, 8: 2})

eps=1210
	minpts=3
	 Counter({1: 4138, -1: 1801, 2: 16, 3: 11, 6: 8, 8: 6, 5: 5, 10: 4, 4: 4, 7: 4, 9: 3})
	minpts=4
	 Counter({1: 4039, -1: 1920, 6: 11, 2: 11, 5: 8, 4: 5, 3: 4, 7: 2})

eps=1220
	minpts=3
	 Counter({1: 4214, -1: 1736, 2: 16, 7: 7, 4: 5, 5: 5, 10: 4, 6: 4, 3: 3, 8: 3, 9: 3})
	minpts=4
	 Counter({1: 4132, -1: 1833, 5: 11, 7: 7, 2: 5, 4: 5, 6: 4, 3: 3})

eps=1230
	minpts=3
	 Counter({1: 4262, -1: 1677, 2: 16, 9: 12, 3: 6, 6: 5, 7: 5, 5: 4, 8: 4, 4: 3, 10: 3, 11: 3})
	minpts=4
	 Counter({1: 4185, -1: 1775, 7: 12, 5: 11, 2: 5, 4: 5, 6: 4, 3: 3})

eps=1240
	minpts=3
	 Counter({1: 4317, -1: 1617, 2: 19, 9: 13, 3: 7, 6: 5, 7: 5, 5: 4, 8: 4, 4: 3, 10: 3, 11: 3})
	minpts=4
	 Counter({1: 4253, -1: 1704, 4: 18, 6: 13, 2: 5, 5: 4, 3: 3})

eps=1250
	minpts=3
	 Counter({1: 4379, -1: 1558, 2: 1

Trying eps=[1100, 1200] diff 10

In [15]:
for eps in range(1100, 1200, 10):
    print('\neps={0}'.format(eps))
    for min_pts in range(3, 5):
        print('\tminpts={0}'.format(min_pts))
        fashion_db = DBSCAN(X=fashion_sub, eps=eps, min_pts=min_pts, distances=d, precomputed=True)
        fashion_db.fit()
        print('\t', Counter(fashion_db.lable))


eps=1100
	minpts=3
	 Counter({1: 3320, -1: 2577, 2: 30, 7: 9, 9: 7, 5: 5, 11: 5, 13: 5, 17: 5, 6: 4, 15: 4, 16: 4, 12: 4, 3: 3, 4: 3, 8: 3, 10: 3, 14: 3, 18: 3, 19: 3})
	minpts=4
	 Counter({1: 3200, -1: 2757, 2: 20, 4: 7, 5: 5, 3: 3, 6: 3, 8: 3, 7: 2})

eps=1110
	minpts=3
	 Counter({1: 3394, -1: 2498, 2: 42, 7: 8, 5: 5, 9: 5, 11: 5, 16: 5, 6: 4, 13: 4, 12: 4, 14: 4, 10: 4, 3: 3, 4: 3, 8: 3, 15: 3, 17: 3, 18: 3})
	minpts=4
	 Counter({1: 3294, -1: 2656, 2: 29, 3: 8, 4: 5, 5: 3, 7: 3, 6: 2})

eps=1120
	minpts=3
	 Counter({1: 3460, -1: 2424, 3: 51, 7: 8, 5: 7, 2: 6, 9: 5, 10: 5, 11: 5, 14: 5, 6: 4, 12: 4, 13: 4, 4: 3, 8: 3, 15: 3, 16: 3})
	minpts=4
	 Counter({1: 3367, -1: 2563, 2: 39, 4: 8, 3: 7, 5: 5, 6: 3, 7: 3, 9: 3, 8: 2})

eps=1130
	minpts=3
	 Counter({1: 3519, -1: 2364, 3: 52, 4: 9, 5: 8, 2: 6, 10: 6, 7: 5, 8: 5, 9: 5, 12: 5, 11: 4, 6: 3, 13: 3, 14: 3, 15: 3})
	minpts=4
	 Counter({1: 3419, -1: 2499, 2: 39, 3: 9, 5: 8, 6: 5, 11: 5, 4: 4, 9: 4, 7: 3, 8: 3, 10: 2})

eps=1140
	minpts=3


Best result: eps=1150

## 20NG

In [3]:
news_train = fetch_20newsgroups(
    data_home='../data/20newsgroups/', 
    subset='train')

In [4]:
vectorizer = TfidfVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(news_train.data)

In [5]:
d = euclidean_distances(feature_matrix)

In [6]:
np.mean(np.mean(np.sort(d, axis=1)[:, 1:5], axis=1))

1.0961815288268209

In [11]:
for _eps in range(7, 14):
    eps = _eps/10
    print('\neps={0}'.format(eps))
    for min_pts in range(3, 5):
        print('\tminpts={0}'.format(min_pts))
        db_20 = DBSCAN(X=feature_matrix, eps=eps, min_pts=min_pts, distances=d, precomputed=True)
        db_20.fit()
        c = Counter(db_20.lable)
        print('\tcluster:', len(c), '\n\t', c)


eps=0.7
	minpts=3
	cluster: 89 
	 Counter({-1: 10944, 1: 11, 3: 10, 8: 10, 54: 9, 35: 8, 32: 7, 4: 6, 68: 6, 28: 6, 73: 6, 46: 6, 48: 6, 55: 6, 9: 5, 13: 5, 23: 5, 24: 5, 25: 5, 37: 5, 40: 5, 88: 5, 78: 5, 85: 5, 2: 4, 5: 4, 6: 4, 27: 4, 14: 4, 15: 4, 19: 4, 36: 4, 29: 4, 30: 4, 31: 4, 33: 4, 38: 4, 39: 4, 72: 4, 41: 4, 42: 4, 45: 4, 47: 4, 52: 4, 53: 4, 58: 4, 65: 4, 70: 4, 67: 4, 69: 4, 77: 4, 86: 4, 7: 3, 10: 3, 11: 3, 12: 3, 16: 3, 17: 3, 18: 3, 20: 3, 21: 3, 22: 3, 26: 3, 34: 3, 43: 3, 44: 3, 49: 3, 50: 3, 51: 3, 56: 3, 57: 3, 59: 3, 60: 3, 61: 3, 62: 3, 63: 3, 64: 3, 66: 3, 71: 3, 74: 3, 75: 3, 76: 3, 79: 3, 80: 3, 81: 3, 82: 3, 83: 3, 84: 3, 87: 3})
	minpts=4
	cluster: 26 
	 Counter({-1: 11177, 2: 10, 4: 10, 1: 9, 20: 9, 12: 8, 11: 7, 24: 6, 8: 6, 21: 6, 5: 5, 6: 5, 7: 5, 3: 4, 9: 4, 10: 4, 14: 4, 15: 4, 16: 4, 17: 4, 18: 4, 19: 4, 22: 4, 23: 4, 25: 4, 13: 3})

eps=0.8
	minpts=3
	cluster: 203 
	 Counter({-1: 10466, 1: 14, 19: 14, 8: 11, 11: 10, 67: 10, 5: 9, 21: 9, 14: 8, 71: 8

In [26]:
for _eps in range(108, 113):
    eps = _eps/100
    print('\neps={0} minpts=3'.format(eps))
    db_20 = DBSCAN(X=feature_matrix, eps=eps, min_pts=3, distances=d, precomputed=True)
    db_20.fit()
    c = Counter(db_20.lable)
    print('\tcluster:', len(c), '\n\t', c)


eps=1.08 minpts=3
	cluster: 579 
	 Counter({-1: 5847, 3: 538, 16: 272, 8: 265, 38: 213, 35: 163, 15: 111, 54: 94, 66: 83, 89: 80, 71: 76, 53: 70, 4: 58, 33: 55, 162: 47, 153: 40, 23: 40, 39: 39, 27: 35, 221: 34, 9: 29, 144: 29, 263: 28, 55: 27, 199: 24, 178: 21, 220: 20, 83: 19, 117: 19, 152: 19, 50: 18, 181: 18, 47: 17, 253: 17, 139: 17, 168: 17, 177: 17, 214: 17, 20: 16, 92: 16, 347: 16, 37: 15, 87: 15, 119: 15, 121: 15, 133: 15, 195: 15, 262: 15, 116: 14, 184: 14, 205: 14, 323: 14, 354: 14, 7: 13, 13: 13, 36: 13, 46: 13, 51: 13, 75: 13, 96: 13, 146: 13, 196: 13, 308: 13, 14: 12, 61: 12, 295: 12, 105: 12, 156: 12, 169: 12, 40: 11, 265: 11, 397: 11, 73: 11, 189: 11, 210: 11, 212: 11, 223: 11, 474: 11, 247: 11, 330: 11, 18: 10, 48: 10, 102: 10, 131: 10, 151: 10, 158: 10, 237: 10, 261: 10, 338: 10, 28: 9, 127: 9, 135: 9, 182: 9, 183: 9, 289: 9, 386: 9, 374: 9, 328: 9, 339: 9, 341: 9, 356: 9, 502: 9, 42: 8, 58: 8, 219: 8, 62: 8, 65: 8, 147: 8, 160: 8, 172: 8, 318: 8, 325: 8, 236: 8, 242

Best eps: 1.1

# Housing

In [3]:
housing_df = np.loadtxt('../data/housing/housing_train.txt')

In [5]:
d = euclidean_distances(housing_df)

In [26]:
sorted(d[0])

[0.0,
 16.562808495238958,
 17.094983236391087,
 18.401273906445255,
 19.105558205162904,
 19.46559434386876,
 19.859987205204476,
 19.91377586075829,
 20.40566434926582,
 22.3851209757722,
 22.39047274489731,
 23.59913472319639,
 23.98188573345461,
 24.262361915792393,
 25.010193366268155,
 25.05791617243707,
 25.235419501137095,
 25.237874417953392,
 25.28168297882082,
 25.395256586614607,
 25.475955285728393,
 25.65806801690352,
 25.660757256361844,
 26.34651389740346,
 26.57760660375637,
 26.640995180736056,
 26.74128069507041,
 26.750364365977315,
 26.84115625382764,
 26.882128600644073,
 26.923743956173976,
 27.127663451680327,
 27.463689117086346,
 27.676324362269337,
 27.860953522198788,
 27.980724437412018,
 28.30960221203298,
 28.319929319022567,
 28.730556905120704,
 28.759243659012153,
 30.088552166064463,
 30.190135069462865,
 31.122366135415927,
 31.192817012644465,
 31.226332928585585,
 31.288799704469017,
 31.399966104479326,
 31.523613190655745,
 31.843643724428908,
 3

In [31]:
np.mean(np.mean(np.sort(d, axis=1)[:, 2:180], axis=1))

104.85213271559891

In [32]:
for eps in range(80, 120, 5):
    print('\neps={0}'.format(eps))
    for min_pts in range(3, 5):
        print('\tminpts={0}'.format(min_pts))
        housing_db = DBSCAN(X=housing_df, eps=eps, min_pts=min_pts, distances=d, precomputed=True)
        housing_db.fit()
        c = Counter(housing_db.lable)
        print('\tcluster:', len(c), '\n\t', c)


eps=80
	minpts=3
	cluster: 4 
	 Counter({1: 320, 2: 86, 3: 25, -1: 2})
	minpts=4
	cluster: 4 
	 Counter({1: 318, 2: 86, 3: 25, -1: 4})

eps=85
	minpts=3
	cluster: 4 
	 Counter({1: 320, 2: 86, 3: 25, -1: 2})
	minpts=4
	cluster: 4 
	 Counter({1: 320, 2: 86, 3: 25, -1: 2})

eps=90
	minpts=3
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})
	minpts=4
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})

eps=95
	minpts=3
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})
	minpts=4
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})

eps=100
	minpts=3
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})
	minpts=4
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})

eps=105
	minpts=3
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})
	minpts=4
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})

eps=110
	minpts=3
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})
	minpts=4
	cluster: 3 
	 Counter({1: 322, 2: 86, 3: 25})

eps=115
	minpts=3
	cluster: 2 
	 Counter({1: 322, 2: 111})
	minpts=4
	cluster: 2 
	 Counter({1: 322, 2: 111})


Eps 95-105 give good results.